In [1]:
import numpy as np
import cv2

try:
    from PIL import Image
except ImportError:
    import Image

In [2]:
# Read a sample image to experiment on
img = cv2.imread('100082743.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [3]:
row_idx = 0
column_idx = 0
pixel_data = np.zeros([len(hsv), len(hsv[0]), len(hsv[0][0])+1])
print(hsv.shape)
print(pixel_data.shape)
for row in hsv:
    for pixel in row:
        pixel_data[row_idx][column_idx] = [pixel[1],pixel[2],column_idx,row_idx]
        column_idx+=1
    column_idx = 0
    row_idx+=1
pixel_data = pixel_data.reshape(len(pixel_data)*len(pixel_data[0]), len(pixel_data[0][0]))

(386, 298, 3)
(386, 298, 4)


In [4]:
means = np.mean(pixel_data, axis=0)
stds = np.std(pixel_data, axis=0)
normed_pixels = []
for row in pixel_data:
    normed_pixels.append((row-means)/stds)
np.mean(normed_pixels, axis=0)

array([  5.73353034e-14,  -1.41890543e-14,   2.89552898e-20,
         3.74372593e-17])

In [5]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(normed_pixels)

In [6]:
kmeans.cluster_centers_

array([[ -8.51038903e-02,   3.31276740e-02,   4.33720122e-03,
          1.04248021e-02],
       [  1.01547814e+01,  -3.95286613e+00,  -5.17524287e-01,
         -1.24391007e+00]])

In [ ]:
red = [0, 0, 255]
for point in normed_pixels:
    if kmeans.predict([point]) == 1:
        if point[2]*stds[2]+means[2] > len(img)-2:
            xcoord = len(img)-1
        else:
            xcoord = point[2]*stds[2]+means[2]
        if point[3]*stds[3]+means[3] > len(img[0])-2:
            ycoord = len(img[0])-1
        else:
            ycoord = point[3]*stds[3]+means[3]
        img[int(round(ycoord)), int(round(xcoord))] = red
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()    